In [13]:
#7.1 blackjack

import random
from abc import ABCMeta, abstractmethod
class Card:
    def __init__(self, num, suit):
        self.num = num
        self.suit= suit
    def __repr__(self):
        num = self.num
        if self.num == 11:
            num = 'J'
        elif self.num == 12:
            num = 'Q'
        elif self.num == 13:
            num = 'K'
        elif self.num == 14:
            num = 'A'
        return "{0} of {1}".format(num, self.suit)
        
class Deck:
    def __init__(self):
        SUITS = ['hearts', 'diamonds', 'clubs', 'spades']
        NUMBERS = [2,3,4,5,6,7,8,9,10,11,12,13,14]  
        self.cards = []
        self.idx = 0
        for suit in SUITS:
            for number in NUMBERS:
                self.cards.append(Card(number, suit))
        random.shuffle(self.cards)
        
    def deal(self):
        card = self.cards[self.idx]
        self.idx+=1
        return card
    
class Hand(metaclass = ABCMeta):
    def get_value(self):
        pass
    def add_card(self):
        pass
    
class BlackJackHand(Hand):
    def __init__(self):
        self.cards = []
        self.val = 0
        
    def update_value(self):
        card = self.cards[-1]
        #special cases - 11-13 - consider as 10
        if card.num <=10:
            self.val += card.num
        elif card.num > 10 and card.num <= 13:
            self.val += 10
        else:
            #14 consider as 1 or 11
            if self.val + 11 > 21:
                self.val += 1
            else:
                self.val += 11
        return self.val
    
    def get_value(self):
        return self.val
    
    def add_card(self,card):
        self.cards.append(card)
        self.update_value()
        
    def is_bust(self):
        if self.val > 21:
            return True
        return False
    
    def is_blackjack(self):
        if self.val == 21 and len(self.cards) == 2:
            if (self.cards[0].num == 14 and (self.cards[1].num >=11 and self.cards[1].num <= 13) or 
                self.cards[1].num == 14 and (self.cards[0].num >=11 and self.cards[0].num <= 13)):
                return True
        return False
    
class BlackJackTable:
    def __init__(self, hand, deck):
        self.hands = hand
        self.deck = deck

In [14]:
deck = Deck()

In [15]:
blackjackhand = BlackJackHand()

In [16]:
blackjackhand.add_card(deck.deal())

In [17]:
print(blackjackhand.get_value())

2


In [18]:
blackjackhand.cards

[2 of clubs]

In [19]:
blackjackhand.is_blackjack()

False

In [20]:
blackjackhand.is_bust()

False

In [30]:
def menu():

    USER_CHOICE = """h to hit\n s to stand\n"""
    CONTINUE = """play again? y/n\n"""


    while True:
        hand = BlackJackHand()
        deck = Deck()
        table = BlackJackTable(hand,deck)
        table.hands.add_card(table.deck.deal())
        table.hands.add_card(table.deck.deal())
        user_choice = ""
        while not table.hands.is_bust() and table.hands.get_value()<21 and user_choice !='s':
            print('current hand\n value:{}\n cards:{}'.format(table.hands.get_value(),table.hands.cards))
            if table.hands.is_blackjack():
                print("Blackjack! player wins")
                break
            else:
                user_choice = input(USER_CHOICE)
                while user_choice != 'h' or user_choice !='s':
                    if user_choice == 'h':
                        print('player hits')
                        table.hands.add_card(table.deck.deal())
                        print('current value of hand : ', table.hands.get_value())
                        break
                    elif user_choice == 's':
                        print('player stands')
                        print('final value of hand: ', table.hands.get_value())
                        break
                    else:
                        print('command not recognized')
                        user_choice = input(USER_CHOICE)
        if table.hands.is_bust():
            print('player busts!')
        elif table.hands.get_value()==21:
            print('player wins with 21!')
        continue_choice = input(CONTINUE)
        while continue_choice != 'y' or continue_choice != 'n':
            if continue_choice =='n':
                return
            elif continue_choice =='y':
                break
            else:
                print('command not recognized')
                continue_choice = input(CONTINUE)

In [108]:
#7.2
"""
use priority queue of responder, manager, and directors to assign work
worker will be baseclass, rest will inherit
make a call class with priority restraint
make a scheduler with p queue as class 
"""
import queue
import random
class Scheduler:
    def __init__(self, responders, managers, directors):
        self.available_workers = [[Responder(str(i)) for i in range(0,responders)],
                                  [Mananger(str(i)) for i in range(0,managers)],
                                  [Director(str(i)) for i in range(0,directors)]] #q of workers based on worker rank
        self.busy_workers =[] #list of ppl working
        self.incoming_calls=[]
        self.finished_calls=[]
    def schedule_call(self, call):
        #find the next available person that can handle a call and assign 
        if call.priority <=5:
            if len(self.available_workers[0]) > 0:
                worker = self.available_workers[0].pop(0)
                print('assigning responder')
            elif len(self.available_workers[0]) == 0 and len(self.available_workers[1]) > 0:
                worker = self.available_workers[1].pop(0)
                print('assigning manager')
            elif len(self.available_workers[0]) == 0 and len(self.available_workers[1]) == 0 and len(self.available_workers[2]) > 0:
                worker = self.available_workers[2].pop(0)
                print('assigning director')
            else:
                print('no workers available at the moment!')
                return False
        elif call.priority > 5 and call.priority <=10:
            if len(self.available_workers[1]) > 0:
                worker = self.available_workers[1].pop(0)
                print('assigning manager')
            elif len(self.available_workers[1]) == 0 and len(self.available_workers[2]) > 0:
                worker = self.available_workers[2].pop(0)
                print('assigning director')
            else:
                print('no workers available at the moment!')
                return False
        else:
            if len(self.available_workers[2]) > 0:
                worker = self.available_workers[2].pop(0)
                print('assigning director')
            else:
                print('no workers available at the moment!')
                return False
        worker.call = call
        self.busy_workers.append(worker)
        return True
    def update_status(self):
        i = 0
        length = len(self.busy_workers)
        while i < length:
            worker = self.busy_workers.pop(0)
            worker.work()
            if worker.current_call_duration >= worker.call.duration:
                self.finished_calls.append(worker.call)
                worker.call = None
                worker.current_call_duration = 0
                worker.busy = False
                if worker.rank == 5:
                    self.available_workers[0].append(worker)
                elif worker.rank == 10:
                    self.available_workers[1].append(worker)
                else:
                    self.available_workers[2].append(worker)
            else:
                self.busy_workers.append(worker)
            i += 1
    def receive_call(self):
        self.incoming_calls.append(Call(random.randint(1,15), random.randint(0,40), 'someone'))
    
class Call:
    def __init__(self, priority, duration, caller):
        self.priority = priority
        #call resolved after duration
        self.duration = duration
        self.caller = caller
    
class Worker:
    def __init__(self, name, title, rank):
        self.name = name
        self.title = title
        self.rank = rank
        self.busy= False
        self.current_call_duration = 0
        self.call = None
    def can_handle_call(self, call):
        if call.priority <= self.rank:
            return True
        return False
    
    def work(self):
        self.current_call_duration +=1
        
class Responder(Worker):
    def __init__(self, name):
        super().__init__(name, 'Responder', 5)
class Mananger(Worker):
    def __init__(self, name):
        super().__init__(name, 'Manager', 10)
class Director(Worker):
    def __init__(self, name):
        super().__init__(name, 'Director', 15)

In [109]:
me = Director('me')

In [110]:
me.name

'me'

In [111]:
me.rank

15

In [112]:
me.title

'Director'

In [113]:
me.current_call_duration

0

In [114]:
me.work()

In [115]:
me.current_call_duration

1

In [116]:
call = Call(5,10,'someone')

In [117]:
call.caller

'someone'

In [118]:
scheduler = Scheduler(10,3,1)

In [119]:
scheduler.available_workers

[[<__main__.Responder at 0x1e63b745b00>,

In [120]:
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()
scheduler.receive_call()

In [121]:
for i in range(0, 8):
    print(scheduler.incoming_calls[i].priority)

15
14
4
1
15
13
9
11


In [122]:
 while len(scheduler.incoming_calls)>0:
        current_call = scheduler.incoming_calls[0]
        if scheduler.schedule_call(current_call):
            scheduler.incoming_calls.pop(0)
        else:
            print('waiting for next available worker')
        scheduler.update_status()
        print(scheduler.busy_workers)

assigning director
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
no workers available at the moment!
waiting for next available worker
n

In [102]:
scheduler.incoming_calls

[]

In [126]:
#7.3
#juke box

class JukeBox:
    def __init__(self):
        self.CDs = []
        self.current_CD = None
        self.user = None
        self.selector = None #interact with current cd
        self.playlist = []
    def __repr__(self):
        #used for display
        return """currently playing : {0}\n
                By : {1} \n
                From: {2}""".format(self.get_song(), self.selector.CD.get_artist(), self.selector.CD.get_album())
    
    def select_CD(self, CD):
        if self.selector :
            self.CDs.append(self.selector.eject_CD)
        self.selector.CD = CD
        self.CDs.remove(CD)
    
    def get_song(self):
        #used for repr
        return self.selector.get_curr_song()
    
    def search_by_artist(self, artist):
        #search by artist, return CDs with artist name
        ret = []
        for cd in self.CDs:
            if cd.artist == artist:
                ret.append(cd)
        return ret
    
    def search_by_song_name(self, song_name):
        #search by song_name, return CD with index to specific song
        ret = []
        for cd in self.CDs:
            for idx, song in enumerate(cd.song_list):
                if song[0] == song_name:
                    ret.append((cd,idx))
        return ret
    
    def search_by_album_name(self, album_name):
        #search by album name return cd\
        ret = []
        for cd in self.CDs:
            if cd.album == album_name:
                ret.append(cd)
        return ret
        
    def create_playlist(self, song_names):
        #create list of songs
        for songs in song_names:
            for result in search_by_song_name(songs):
                self.playlist.append(result)
        
    def play_playlist(self):
        for cd, idx in self.playlist:
            select_CD(cd)
            self.selector.play_specific_song(idx)
    
class CD:
    def __init__(self, artist, album, song_list):
        self.artist = artist
        self.album = album
        self.song_list = song_list #lets say song list is a list of tuples of (title, songlength)
    def __repr__(self):
        return "{0} by {1}".format(self.album, self.artist)
    def get_artist(self):
        return self.artist
    def get_album(self):
        return self.album

class Selector:
    #this interacts with the current cd in the jukebox
    def __init__(self):
        self.CD = None
        self.curr_song = None
        self.song_idx = None
        
    def load_CD(CD):
        self.CD = CD
        
    def eject_CD(self):
        ret = self.CD
        self.CD =None
        self.curr_song = None
        self.song_idx = None
        return ret
    
    def play(self):
        self.curr_song = self.CD.song_list[0]
        self.song_idx = 0
        
    def stop(self):
        self.curr_song = None
        self.song_idx = None
    
    def get_curr_song(self):
        return self.curr_song
    
    def play_specific_song(self, index):
        self.song_idx = index
        self.curr_song = self.song_idx[index]
    
    def play_next_song(self):
        if self.song_idx == len(self.CD.song_list)-1:
            self.song_idx = 0
        else:
            self.song_idx+=1
        self.curr_song = self.CD.song_list[self.song_idx]
        
    def play_prev_song(self):
        if self.song_idx == 0:
            self.song_idx = len(self.CD.song_list)-1
        else:
            self.song_idx -= 1
        self.curr_song = self.CD.song_list[self.song_idx]

In [ ]:
#7.4
#parking lot

